In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import autocorrelation_plot
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA #p=1, d=1, q=0 or 1
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pandas.tseries.offsets import DateOffset
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
main_df=pd.read_csv('/kaggle/input/perrin-freres-monthly-champagne-sales/Perrin Freres monthly champagne sales millions.csv')
main_df.head()

In [ ]:
main_df.tail()

In [ ]:
main_df.drop(105,axis=0,inplace=True)
main_df.drop(106,axis=0,inplace=True)
main_df.tail()

In [ ]:
main_df.columns=["Month","Sales"]
main_df['Month']=pd.to_datetime(df['Month'])
df= main_df.set_index('Month')
df.head()

In [ ]:
df.describe()

In [ ]:
df.plot()

In [ ]:
test_result=adfuller(df['Sales'])
def adfuller_test(sales):
    result=adfuller(sales)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("It is stationary")
    else:
        print("It is non-stationary ")
adfuller_test(df['Sales'])        

In [ ]:
df['Sales First Difference'] = df['Sales'] - df['Sales'].shift(1)
df['Sales'].shift(1)
df['Seasonal First Difference']=df['Sales']-df['Sales'].shift(12)
df.head(14)

In [ ]:
adfuller_test(df['Seasonal First Difference'].dropna())

In [ ]:
df['Seasonal First Difference'].plot()

In [ ]:
autocorrelation_plot(df['Sales'])
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df['Seasonal First Difference'].iloc[13:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df['Seasonal First Difference'].iloc[13:],lags=40,ax=ax2)

In [ ]:
model=ARIMA(df['Sales'],order=(1,2,1))
model_fit=model.fit()
model_fit.summary()

In [ ]:
df['forecast']=model_fit.predict(start=90,end=103,dynamic=True)
df[['Sales','forecast']].plot(figsize=(12,8))

In [ ]:
model=sm.tsa.statespace.SARIMAX(df['Sales'],order=(1, 1, 1),seasonal_order=(1,1,1,12))
results=model.fit()
df['forecast']=results.predict(start=90,end=106,dynamic=True)
df[['Sales','forecast']].plot(figsize=(12,8))

In [ ]:
future_dates=[df.index[-1]+ DateOffset(months=x)for x in range(0,48)]
future_datest_df=pd.DataFrame(index=future_dates[1:],columns=df.columns)
future_datest_df.tail()
future_df=pd.concat([df,future_datest_df])
future_df['forecast'] = results.predict(start = 104, end = 152, dynamic= True)  

In [ ]:
future_df['Sales'].isna().sum()

In [ ]:
future_df[['Sales', 'forecast']].plot(figsize=(12, 8)) 

In [ ]:
result = seasonal_decompose(df['Sales'], model='multiplicative')
result.plot()

In [ ]:
result = seasonal_decompose(df['Sales'], model='additive')
result.plot()

In [ ]:
sns.heatmap(df.corr(),annot = True)

In [ ]:
sns.heatmap(main_df.corr(),annot = True)